<a href="https://colab.research.google.com/github/Srikanth149/Srikanth149/blob/master/LORA_Distilbert_pre_post_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType


In [3]:
positive_samples = [
    "I absolutely loved this movie!",
    "Fantastic film and brilliant acting.",
    "Great direction and powerful story.",
    "It was a wonderful experience.",
    "Loved every moment of it.",
    "A masterpiece worth watching again.",
    "The soundtrack and visuals were stunning.",
    "An emotional and gripping narrative.",
    "Highly recommend this movie to everyone.",
    "Outstanding performance by the lead actor."
]


negative_samples = [
    "I hated this movie so much.",
    "Terrible acting and weak plot.",
    "It was a complete waste of time.",
    "Not worth watching at all.",
    "Worst direction I’ve ever seen.",
    "Painful to sit through.",
    "The story made no sense.",
    "Horrible visuals and bad music.",
    "Regret watching this film.",
    "Everything felt forced and boring."
]

# Replicate to increase total number of samples
samples = {
    "sentence": positive_samples * 5 + negative_samples * 5, # 50 positive + 50 negative
    "label": [1]*50 + [0]*50
}

from datasets import Dataset
import random


In [4]:
dataset = Dataset.from_dict(samples).shuffle(seed=42)
split_dataset = dataset.train_test_split(test_size=0.3)



In [5]:
model_id = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
def tokenize(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = split_dataset.map(tokenize)


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_base = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2).to(device)



model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.1,
    target_modules=["q_lin","v_lin"],
    bias="none"
)

model = get_peft_model(model, lora_config)


In [10]:
training_args = TrainingArguments(
    output_dir="./lora-output",
    per_device_train_batch_size=8,
    learning_rate=5e-4,
    num_train_epochs=10,
    logging_steps=10,
    report_to="none",
    save_strategy="no"
)



In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)



<ipython-input-11-ce0548f1ac28>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
model.train()
trainer.train()


Step,Training Loss
10,0.577900
20,0.263800
30,0.011100
40,0.000200
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000


TrainOutput(global_step=90, training_loss=0.09478120070044598, metrics={'train_runtime': 8.5406, 'train_samples_per_second': 81.961, 'train_steps_per_second': 10.538, 'total_flos': 23579396198400.0, 'train_loss': 0.09478120070044598, 'epoch': 10.0})

In [13]:
sample_text = "Great story, direction, and cast!"
inputs = tokenizer(sample_text, return_tensors="pt").to(device)


In [14]:
model_base.eval()
with torch.no_grad():
    logits_before = model_base(**inputs).logits
    pred_before = logits_before.argmax(dim=-1).item()


In [15]:
model.eval()
with torch.no_grad():
    logits_after = model(**inputs).logits
    pred_after = logits_after.argmax(dim=-1).item()


In [18]:
print("\n🔍 Input Sentence:", sample_text)
print("Logits BEFORE LoRA Tuning:", logits_before)
print("Prediction BEFORE Tuning:", "Positive ✅" if pred_before == 1 else "Negative ❌")
print("Logits AFTER LoRA Tuning :", logits_after)
print("Prediction AFTER Tuning :", "Positive ✅" if pred_after == 1 else "Negative ❌")




🔍 Input Sentence: Great story, direction, and cast!
Logits BEFORE LoRA Tuning: tensor([[-0.0883, -0.0571]], device='cuda:0')
Prediction BEFORE Tuning: Positive ✅
Logits AFTER LoRA Tuning : tensor([[-6.3407,  6.5950]], device='cuda:0')
Prediction AFTER Tuning : Positive ✅
